In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 23.7MB/s 
     |████████████████████████████████| 92kB 8.6MB/s 
     |████████████████████████████████| 3.8MB 43.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [9]:
# 토큰화를 위한 모듈
from konlpy.tag import Okt
import pandas as pd
import re

pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

# 네이버 웹툰 가져오기
naver_wt = pd.read_csv('./drive/My Drive/acyn/naver_webtoon.csv', delimiter = '\t', header=None, encoding='utf-8')
naver_wt = pd.DataFrame({'name' : naver_wt[0].str.split(',').str[0], 'intro' : naver_wt[0] }, columns = ['name', 'intro'])

# 다음 웹툰 가져오기
daum_wt = pd.read_csv('./drive/My Drive/acyn/daum_webtoon.csv', delimiter = '\t', header=None, encoding='utf-8')
daum_wt = pd.DataFrame({'name' : daum_wt[0].str.split(',').str[0], 'intro' : daum_wt[0]}, columns = ['name', 'intro'])

# 네이버 웹소설 가져오기
naver_nvl = pd.read_csv('./drive/My Drive/acyn/naver_novel.csv', header=None, encoding='utf-8')
naver_nvl = pd.DataFrame({'name' : naver_nvl[0], 'intro' : naver_nvl[1]}, columns = ['name', 'intro'])

# 넷플릭스 가져오기
netflix = pd.read_csv('./drive/My Drive/acyn/netflix.csv')
netflix = netflix.dropna(axis=0)
netflix = netflix.drop(['Unnamed: 0'], axis=1)
netflix_plot = netflix['netflix']

# stopword 제거
sw = ['.', ',', '"','"&', '"""','\n', '이', '가', '을', '를', '는', '은', '과', '의', '로','에서','에', '도','에게','와', '인']

# 네이버 웹툰 토큰화 및 stopword 제거
for intro_num in range(len(naver_wt.intro)):
    words = re.compile("[^\w]").sub(' ', naver_wt.intro[intro_num])
    words = Okt().morphs(words)
    sw_removed = []
    for i in words:
       if i.lower() not in sw:
           sw_removed.append(i)
    naver_wt.intro[intro_num] = sw_removed

# 다음 웹툰 토큰화 및 stopword 제거
for intro_num in range(len(daum_wt.intro)):
    words = re.compile("[^\w]").sub(' ', daum_wt.intro[intro_num])
    words = Okt().morphs(words)
    sw_removed = []
    for i in words:
       if i.lower() not in sw:
           sw_removed.append(i)
    daum_wt.intro[intro_num] = sw_removed

# 네이버 웹소설 토큰화 및 stopword 제거
for intro_num in range(len(naver_nvl.intro)):
    words = re.compile("[^\w]").sub(' ', naver_nvl.intro[intro_num])
    words = Okt().morphs(words)
    sw_removed = []
    for i in words:
       if i.lower() not in sw:
           sw_removed.append(i)
    naver_nvl.intro[intro_num] = sw_removed

# 넷플릭스 토큰화 및 stopword 제거
toknized_plots_list = []
for plot in netflix_plot:
  words_list_for_one_plot = Okt().morphs(plot)
  sw_removed = []
  for i in words_list_for_one_plot:
      if i.lower() not in sw:
          sw_removed.append(i)
  toknized_plots_list.append(sw_removed)
  
netflix["tokenized_plot"] = toknized_plots_list

In [7]:
# 사용자 입력받기
input_wd = input()

naver_wt_recommend = []
daum_wt_recommend = []
naver_nvl_recommend = []
netflix_recommend = []

# 네이버 웹툰에서 입력 관련 컨텐츠 뽑기
for intro_num in range(len(naver_wt.intro)):
    if input_wd in naver_wt.intro[intro_num] :
        naver_wt_recommend.append(naver_wt.name[intro_num])

# 다음 웹툰에서 입력 관련 컨텐츠 뽑기
for intro_num in range(len(daum_wt.intro)):
    if input_wd in daum_wt.intro[intro_num] :
        daum_wt_recommend.append(daum_wt.name[intro_num])

# 네이버 웹소설에서 입력 관련 컨텐츠 뽑기
for intro_num in range(len(naver_nvl.intro)):
    if input_wd in naver_nvl.intro[intro_num] :
        naver_nvl_recommend.append(naver_nvl.name[intro_num])

# 넷플릭스에서 입력 관련 컨텐츠 뽑기
for video in netflix.iterrows():
  tokenized = video[1][5]
  for word in tokenized:
      if word == input_wd:
        netflix_recommend.append(video[1][1])

print("추천 네이버 웹툰 :",naver_wt_recommend)
print("추천 다음 웹툰 :",daum_wt_recommend)
print("추천 네이버 웹소설 :", naver_nvl_recommend)
print("추천 넷플릭스 : ", netflix_recommend)


학교
추천 네이버 웹툰 : ['인생존망', '야자괴담', '럭키언럭키', '셧업앤댄스', '커피도둑', '소녀재판', '학교정벌', '신이 담긴 아이', '만찢남녀', 'Here U Are']
추천 다음 웹툰 : ['아싸가 알아버렸다', '이대로 멈출 순 없다', '학교대표', '우리학교 김선생님']
추천 네이버 웹소설 : ['아기가 생겼어요', '표류전쟁', '납치 감금에서 시작되는 우리들의 사바트']
추천 넷플릭스 :  ['꼴찌 마녀 밀드레드', 'The Principal', '학교생활', '핑키 멀링키', '보고싶다', '러브포텐: 순정의 시대', '러브 라이브! School idol project', '요리고교생', '카케구루이', '그린하우스 아카데미', '굿모닝 콜', '걸즈 앤 판처', '그녀의 이름은 난노', '에버 애프터 하이', '엘리트들', '샤를로트', '백스테이지', '선생 김봉두', '가질 수 있다면', '여고괴담 두번째 이야기', '여고괴담 4: 목소리', '트윈스터즈', '협객 소걸아', '더 리허설', '아웃사이더', '루저의 역습', '뉴 가이', '뉴 가이', '키싱 부스', '크래프트', '설득의 신', '로미나', '칠소복', '칠소복', 'Nothing in Return', '네버 렛 미 고', '노바: 미래의 학교', '아적야만동학: 거친 녀석들', '아적야만동학: 거친 녀석들', '아적야만동학: 거친 녀석들', '기적의 학교', '맥팔랜드 USA', '너의 췌장을 먹고 싶어', '나는 거인들을 죽인다', '허리케인 비앙카', '사냥의 계절', '신부님은 골키퍼', '우정의 조건', '한공주', '폴른: 추락천사', '엔더스 게임', '이모 더 뮤지컬', '돈 크라이 마미', '디태치먼트', '조폐국 침입 프로젝트', '코치 카터', '캐리', '브루노와 부츠: 수영장을 부탁해', '블루 마운틴 스테이트: 더 라이즈 오브 사드랜드', "바비와 동생들이 함께하는 '포니 테일'", '합격!', '봄의 멜로디', '내가 널 